# 机器学习工程师纳米学位
## 监督学习
## 项目 2: 搭建一个学生干预系统

欢迎来到机器学习工程师纳米学位的第二个项目！在此文件中，有些示例代码已经提供给你，但你还需要实现更多的功能让项目成功运行。除非有明确要求，你无须修改任何已给出的代码。以**'练习'**开始的标题表示接下来的代码部分中有你必须要实现的功能。每一部分都会有详细的指导，需要实现的部分也会在注释中以**'TODO'**标出。请仔细阅读所有的提示！

除了实现代码外，你还**必须**回答一些与项目和你的实现有关的问题。每一个需要你回答的问题都会以**'问题 X'**为标题。请仔细阅读每个问题，并且在问题后的**'回答'**文字框中写出完整的答案。我们将根据你对问题的回答和撰写代码所实现的功能来对你提交的项目进行评分。

>**提示：**Code 和 Markdown 区域可通过 **Shift + Enter** 快捷键运行。此外，Markdown可以通过双击进入编辑模式。

### 问题 1 - 分类 vs. 回归
*在这个项目中你的任务是找出那些如果不给予帮助，最重可能无法毕业的学生。你觉得这个问题是哪种类型的监督学习问题，是分类问题还是回归问题？为什么？*

**答案: **

我认为这是**分类**问题。因为一个学生在「是否需要干预才能毕业」这个问题上的答案无非「是」与「否」两种，超过某个概率时我们认为需要提供适当干预，当低于该概率时我们认为还不需要介入干预。也就是说学生被分为 2 类：

- 需要干预
- 不需要干预

## 分析数据
运行下面区域的代码以载入学生数据集，以及一些此项目所需的Python库。注意数据集的最后一列`'passed'`是我们的预测的目标（表示学生是毕业了还是没有毕业），其他的列是每个学生的属性。

In [1]:
# 载入所需要的库
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# 载入学生数据集
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### 练习: 分析数据
我们首先通过调查数据，以确定有多少学生的信息，并了解这些学生的毕业率。在下面的代码单元中，你需要完成如下的运算：
- 学生的总数， `n_students`。
- 每个学生的特征总数， `n_features`。
- 毕业的学生的数量， `n_passed`。
- 未毕业的学生的数量， `n_failed`。
- 班级的毕业率， `grad_rate`， 用百分数表示(%)。


In [2]:
# TODO： 计算学生的数量
n_students = len(student_data)  #Better solution???

# TODO： 计算特征数量
n_features = len(student_data.count())   #Better solution???

# TODO： 计算通过的学生数
n_passed = len(student_data[student_data.passed == 'yes'])    #Better solution???

# TODO： 计算未通过的学生数
n_failed = len(student_data[student_data.passed == 'no'])   #Better solution???

# TODO： 计算通过率
grad_rate = float(n_passed) / n_students * 100

# 输出结果
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## 数据准备
在这个部分中，我们将要为建模、训练和测试准备数据
### 识别特征和目标列
你获取的数据中通常都会包含一些非数字的特征，这会导致一些问题，因为大多数的机器学习算法都会期望输入数字特征进行计算。

运行下面的代码单元将学生数据分成特征和目标列看一看他们中是否有非数字特征。

In [3]:
# 提取特征列
feature_cols = list(student_data.columns[:-1])

# 提取目标列 ‘passed’
target_col = student_data.columns[-1] 

# 显示列的列表
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# 将数据分割成特征数据和目标数据（即X_all 和 y_all）
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# 通过打印前5行显示特征信息
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### 预处理特征列

正如你所见，我们这里有几个非数值的列需要做一定的转换！它们中很多是简单的`yes`/`no`，比如`internet`。这些可以合理地转化为`1`/`0`（二元值，binary）值。

其他的列，如`Mjob`和`Fjob`，有两个以上的值，被称为_分类变量（categorical variables）_。处理这样的列的推荐方法是创建和可能值一样多的列（如：`Fjob_teacher`，`Fjob_other`，`Fjob_services`等），然后将其中一个的值设为`1`另外的设为`0`。

这些创建的列有时候叫做 _虚拟变量（dummy variables）_，我们将用[`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies)函数来完成这个转换。运行下面代码单元的代码来完成这里讨论的预处理步骤。

In [4]:
def preprocess_features(X):
    ''' 预处理学生数据，将非数字的二元特征转化成二元值（0或1），将分类的变量转换成虚拟变量
    '''
    
    # 初始化一个用于输出的DataFrame
    output = pd.DataFrame(index = X.index)
    #print output
    #cont = raw_input("continue?")

    # 查看数据的每一个特征列
    for col, col_data in X.iteritems():
        
        # 如果数据是非数字类型，将所有的yes/no替换成1/0
        #print col
        #print col_data
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        #print col_data

        #print '---------'
        # 如果数据类型是类别的（categorical），将它转换成虚拟变量
        if col_data.dtype == object:
            # 例子: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col) 
        #print col_data
        
        # 收集转换后的列
        output = output.join(col_data)
        #cont = raw_input("continue?")
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### 实现: 将数据分成训练集和测试集
现在我们已经将所有的 _分类的（categorical）_ 特征转换成数值了。下一步我们将把数据（包括特征和对应的标签数据）分割成训练集和测试集。在下面的代码单元中，你需要完成下列功能：
- 随机混洗（shuffle）切分数据(`X_all`, `y_all`) 为训练子集和测试子集。
  - 使用300个数据点作为训练集（约75%），使用95个数据点作为测试集（约25%）。
  - 如果可能的话，为你使用的函数设置一个`random_state`。
  - 将结果存储在`X_train`, `X_test`, `y_train`和 `y_test`中。

In [5]:
# TODO：在这里导入你可能需要使用的另外的功能
from sklearn.cross_validation import train_test_split

# TODO：设置训练集的数量
num_train = 300

# TODO：设置测试集的数量
num_test = X_all.shape[0] - num_train

# TODO：把数据集混洗和分割成上面定义的训练集和测试集
X_train ,X_test, y_train, y_test = train_test_split(X_all, y_all,
                                                    train_size=num_train, test_size=num_test,
                                                    random_state=19)

# 显示分割的结果
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## 训练和评价模型
在这个部分，你将选择3个适合这个问题并且在`scikit-learn`中已有的监督学习的模型。首先你需要说明你选择这三个模型的原因，包括这些数据集有哪些特点，每个模型的优点和缺点各是什么。然后，你需要将这些模型用不同大小的训练集（100个数据点，200个数据点，300个数据点）进行训练，并用F<sub>1</sub>的值来衡量。你需要制作三个表，每个表要显示训练集大小，训练时间，预测时间，训练集上的F<sub>1</sub>值和测试集上的F<sub>1</sub>值（每个模型一个表）。

**这是目前** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **里有的监督学习模型，你可以从中选择:**
- Gaussian Naive Bayes (GaussianNB) 朴素贝叶斯
- Decision Trees 决策树
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM) 向量模型机
- Logistic Regression 逻辑回归

### 问题 2 - 应用模型
*列出三个适合这个问题的监督学习算法模型。每一个你选择的模型：*

- 描述一个该模型在真实世界的一个应用场景。（你需要为此做点研究，并给出你的引用出处）
- 这个模型的优势是什么？他什么情况下表现最好？
- 这个模型的缺点是什么？什么条件下它表现很差？
- 根据我们当前数据集的特点，为什么这个模型适合这个问题。

**回答: **

### 数据集的特点是

- 属于**分类**问题，而且是**二分类**（而非多分类）问题
- 数据规模小
- 特征空间小


### 模型1：逻辑回归

#### 该模型在真实世界的一个应用场景

[预测用户对品类的购买偏好](http://tech.meituan.com/intro_to_logistic_regression.html)

#### 优势

- 对分类可能性直接进行建模，而无需事先假设数据分布（周志华《机器学习》 P58）
- 参考[这里](http://t.cn/RcLc47K)
  - 实现简单，广泛的应用于工业问题上；
  - 分类时计算量非常小，速度很快，存储资源低；
  - 便利的观测样本概率分数；

#### 缺点

- 参考[这里](http://t.cn/RcLc47K)
  - 当特征空间很大时，逻辑回归的性能不是很好；
  - 容易**欠拟合**，一般准确度不太高
  - 只能处理两分类问题（在此基础上衍生出来的softmax可以用于多分类），且必须**线性可分**；
  - 对于非线性特征，需要进行转换；

#### 模型为何适合该数据集

- 数据集是二分类问题
- 数据集特征空间小
- 逻辑回归计算量小，速度快，存储资源低，能节省更多资源，更好地满足[项目要求](https://classroom.udacity.com/nanodegrees/nd009/parts/0091345404/modules/544698886575461/lessons/5446988865239847/concepts/54464939410923#)提出的

### 模型2：SVM

#### 该模型在真实世界的一个应用场景

- 文本分类（95年由 Cortes and Vapnik 发表，98 年在文本分类任务中显示出卓越性能，很快成为机器学习主流技术，并直接掀起了「统计学习」在 2000 年前后的高潮）（周志华《机器学习》 P139）

#### 优势

- 以下主要来源于 http://t.cn/RcLc47K
  - 可以解决高维问题，即大型特征空间；
  - 能够处理非线性特征的相互作用；
  - 无需依赖整个数据集，[因此不需要太大内存](http://www.36dsj.com/archives/35477)；
  - 可以提高泛化能力；
- 以下来源于  http://scikit-learn.org/stable/modules/svm.html
  - 维数比样本数多的时候仍有效（！）

#### 缺点

- 以下来源于 http://t.cn/RcLc47K 与 http://www.36dsj.com/archives/35477
  - 当观测样本很多时，效率并不是很高；
  - 对非线性问题没有通用解决方案，有时候很难找到一个合适的核函数；
  - 对缺失数据和噪声敏感；
- 以下来源于  http://scikit-learn.org/stable/modules/svm.html
  - 如果维数比样本数多的话，表现较差

#### 模型为何适合该数据集

- 数据集规模较小
- 维数比样本少（维数超过样本时性能不好，而数据集这一特点可避免该问题）
- 无需依赖整个数据集，不需要太大内存，可以[节省资源](https://classroom.udacity.com/nanodegrees/nd009/parts/0091345404/modules/544698886575461/lessons/5446988865239847/concepts/54464939410923#)

### 模型3：KNN

#### 该模型在真实世界的一个应用场景

- [光学字符识别（OCR）](https://zh.wikipedia.org/wiki/%E6%9C%80%E9%82%BB%E8%BF%91%E6%90%9C%E7%B4%A2)，例如：[手写文字识别](http://scikit-learn.org/stable/modules/neighbors.html)

#### 优势

- 学习速度快
- [非参数学习，可用于决策边界不规则的情况](http://scikit-learn.org/stable/modules/neighbors.html)
- [算法简单直观，易于理解和实现](http://kesmlcv.blogspot.com/2013/08/knn.html)
- [可用于非线性分类](http://t.cn/RcLc47K)
- [对数据没有假设，准确度高，对outlier不敏感](http://t.cn/RcLc47K)

#### 缺点

- 空间开销大
- 使用（预测）时时间开销不如急切学习（eager learning）快，为 $O(n)$ 复杂度（后者为常数复杂度 $O(1)$）
- 容易受到维度灾难的困扰
- K 较小时估计误差（estimation error）较大，即容易过拟合；K 较大时近似误差（approximation error）较大，即容易受到较远的（不相似的）训练点的影响（李航《统计学习方法》P40）

#### 模型为何适合该数据集

- 基于一个假设：特征相似的人成绩相似，因此通过状态与否差异不大
- 数据规模小

### 参考资料

- Web 信息均以链接形式标注
- 周志华《机器学习》，ISBN：9787302423287
- 李航《统计学习方法》，ISBN：9787302275954

### 准备
运行下面的代码单元以初始化三个帮助函数，这三个函数将能够帮你训练和测试你上面所选择的三个监督学习算法。这些函数是：
- `train_classifier` - 输入一个分类器和训练集，用数据来训练这个分类器。
- `predict_labels` - 输入一个训练好的分类器、特征以及一个目标标签，这个函数将帮你做预测并给出F<sub>1</sub>的值.
- `train_predict` - 输入一个分类器以及训练集和测试集，它可以运行`train_clasifier`和`predict_labels`.
 - 这个函数将分别输出训练集的F<sub>1</sub>值和测试集的F<sub>1</sub>值

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' 用训练集训练分类器 '''
    
    # 开始计时，训练分类器，然后停止计时
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' 用训练好的分类器做预测并输出F1值'''
    
    # 开始计时，作出预测，然后停止计时
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # 输出并返回结果
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' 用一个分类器训练和预测，并输出F1值 '''
    
    # 输出分类器名称和训练集大小
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # 训练一个分类器
    train_classifier(clf, X_train, y_train)
    
    # 输出训练和测试的预测结果
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### 练习: 模型评价指标
借助于上面定义的函数，你现在需要导入三个你选择的监督学习模型，然后为每一个模型运行`train_predict`函数。请记住，对于每一个模型你需要在不同大小的训练集（100，200和300）上进行训练和测试。所以，你在下面应该会有9个不同的输出（每个模型都有训练集大小不同的三个输出）。在接下来的代码单元中，你将需要实现以下功能：
- 引入三个你在上面讨论过的监督式学习算法模型。
- 初始化三个模型并将它们存储在`clf_A`， `clf_B` 和 `clf_C`中。
 - 如果可能对每一个模型都设置一个`random_state`。
 - **注意:** 这里先使用每一个模型的默认参数，在接下来的部分中你将需要对某一个模型的参数进行调整。
- 创建不同大小的训练集用来训练每一个模型。
 - *不要再混洗和再分割数据！新的训练集要取自`X_train`和`y_train`.*
- 对于每一个模型要用不同大小的训练集来训练它，然后在测试集上做测试（总共需要9次训练测试）   
**注意:** 在下面的代码单元后面我们提供了三个表用来存储你的结果。

In [7]:
# TODO：从sklearn中引入三个监督学习模型
#from sklearn.cross_validation import KFold
#from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# TODO：初始化三个模型
clf_A = LogisticRegression(random_state=19)
clf_B = SVC(random_state=19)
clf_C = KNeighborsClassifier()

# TODO：设置训练集大小
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO：对每一个分类器和每一个训练集大小运行'train_predict' 
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)

train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0027 seconds
Made predictions in 0.0011 seconds.
F1 score for training set: 0.8092.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7972.
Training a LogisticRegression using a training set size of 200. . .
Trained model in 0.0022 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.8390.
Made predictions in 0.0001 seconds.
F1 score for test set: 0.7832.
Training a LogisticRegression using a training set size of 300. . .
Trained model in 0.0028 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.8266.
Made predictions in 0.0001 seconds.
F1 score for test set: 0.8366.
Training a SVC using a training set size of 100. . .
Trained model in 0.0012 seconds
Made predictions in 0.0007 seconds.
F1 score for training set: 0.8611.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.8765.
Training a SVC using a training set size of 200. . .
Trained mo

### 结果表格
编辑下面的表格看看在[Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables)中如何设计一个表格。你需要把上面的结果记录在表格中。

** 分类器 1 - 逻辑回归**  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0027 | 0.0002 | 0.8092 | 0.7972 |
| 200               | 0.0022 | 0.0001 | 0.8390 | 0.7832 |
| 300               | 0.0028 | 0.0001 | 0.8266 | 0.8366 |


** 分类器 2 - SVM**  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0012 | 0.0008 | 0.8611 | 0.8765 |
| 200               | 0.0037 | 0.0012 | 0.8736 | 0.8516 |
| 300               | 0.0064 | 0.0017 | 0.8702 | 0.8535 |

** 分类器 3 - KNN**  

| 训练集大小 | 训练时间 | 预测时间 (测试) | F1值 (训练) | F1值 (测试) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0004 | 0.0009 | 0.7344 | 0.8027 |
| 200               | 0.0005 | 0.0014 | 0.7971 | 0.8056 |
| 300               | 0.0006 | 0.0019 | 0.8463 | 0.7778 |

## 选择最佳模型
在最后这一部分中，你将从三个监督学习模型中选择一个用在学生数据上的最佳模型。然后你将在最佳模型上用全部的训练集（`X_train`和`y_train`）运行一个网格搜索算法，在这个过程中，你要至少调整一个参数以提高模型的F<sub>1</sub>值（相比于没有调参的模型的分值有所提高）。 

### 问题 3 - 选择最佳模型
*给予你上面做的实验，用一到两段话，向（学校）监事会解释你将选择哪个模型作为最佳的模型。哪个模型在现有的数据，有限的资源、开支和模型表现综合来看是最好的选择？*

**回答: **

按照测试表格看来，我建议选择分类器 2（SVM）作为最佳模型：

1. 从现有的数据看来，尽管分类器 3（KNN）训练时间最短，但其预测时间却比分类器 1 长，其时间和分类器 2 同一个数量级，同时预测准确率远远不如其余 2 个模型
2. 分类器 1（逻辑回归）训练时间和预测时间都比分类器 2（SVM）低，但无论从均值对比（81.53% 与 86.44%）还是中位数对比（81.79% 与 86.57%）来看，预测准确率差别大约有 5% 左右

### 问题 4 - 用通俗的语言解释模型
*用一到两段话，向（学校）监事会用外行也听得懂的话来解释最终模型是如何工作的。你需要解释所选模型的主要特点。例如，这个模型是怎样被训练的，它又是如何做出预测的。避免使用高级的数学或技术术语，不要使用公式或特定的算法名词。*

**回答: **

对于训练数据集中的每一个学生，我们既知道他的各项基本状况，又知道他是否通过考试；那么我们可以用「支持向量机」（SVM）的方法，通过已知的学生数据，在「学生各项基本状况」与「学生是否通过考试」之间建立一个模型。

- 模型在训练时会会找到能**最明显区分**「通过」的学生与「未通过」的学生的「学生各项基本状况」组合，作为区分两类学生的依据。所谓「**最明显区分**」，指的是：这种分类依据下的所有临界点（即差点就可能「通过」或「不通过」的学生）与两类学生（明显「通过」或明显「不通过」）的特征差异最大；在其他分类依据下，临界点与两类学生的特征差异都更小。
- 使用（即预测）时，我们把任一学生的特征（即各项基本状况）输入给模型，模型就能将该生的特征与分类依据比较，判断该生属于「通过」还是「不通过」的类别。

也就是说，分类依据并不「偏袒」或「像」任何一类学生的特征，从而该分类依据下两类学生的差异最大。这就保证了预测的准确率将比较高，而且能够提高对未知数据的预测准确率。

### 练习: 模型调参
细调选择的模型的参数。使用网格搜索（`GridSearchCV`）来至少调整模型的重要参数（至少调整一个），这个参数至少需给出并尝试3个不同的值。你要使用整个训练集来完成这个过程。在接下来的代码单元中，你需要实现以下功能：
- 导入 [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) 和 [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- 创建一个对于这个模型你希望调整参数的字典。
 - 例如: `parameters = {'parameter' : [list of values]}`。
- 初始化你选择的分类器，并将其存储在`clf`中。
- 使用`make_scorer` 创建F<sub>1</sub>评分函数并将其存储在`f1_scorer`中。
 - 需正确设定参数`pos_label`的值！
- 在分类器`clf`上用`f1_scorer` 作为评价函数运行网格搜索,并将结果存储在`grid_obj`中。
- 用训练集(`X_train`, `y_train`)训练grid search object,并将结果存储在`grid_obj`中。

In [8]:
# TODO: 导入 'GridSearchCV' 和 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.naive_bayes import BernoulliNB

# TODO：创建你希望调整的参数列表
parameters = {'gamma': np.power(10.0, [3, 2, 1, 0, -1, -2, -3]), 
              'degree':range(0, 10)}

# TODO：初始化分类器
clf = SVC(random_state=19)

# TODO：用'make_scorer'创建一个f1评分函数
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO：在分类器上使用f1_scorer作为评分函数运行网格搜索
grid_obj = GridSearchCV(clf, param_grid=parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
# TODO：用训练集训练grid search object来寻找最佳参数
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
# 得到预测的结果
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
# 输出经过调参之后的训练集和测试集的F1值
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0060 seconds.
Tuned model has a training F1 score of 0.9798.
Made predictions in 0.0020 seconds.
Tuned model has a testing F1 score of 0.8765.


### 问题 5 - 最终的 F<sub>1</sub> 值
*最终模型的训练和测试的F<sub>1</sub>值是多少？这个值相比于没有调整过参数的模型怎么样？*

**回答: **

最终模型训练的 $F_{1}$ 值是 0.9798，测试的 $F_{1}$ 值是 0.8765。相比没调整过参数的模型，训练的 $F_{1}$ 值大幅上升（相比之前任何一组训练集的训练结果 0.8611、0.8736、0.8702，上升了超过 10%，而且接近 100%的预测准确率），测试的 $F_{1}$ 值上升（比 n=300 的测试集测试结果 0.8535 高，达到之前三组测试集中的最佳测试结果 0.8765）。

> **注意**: 当你写完了所有的代码，并且回答了所有的问题。你就可以把你的 iPython Notebook 导出成 HTML 文件。你可以在菜单栏，这样导出**File -> Download as -> HTML (.html)**把这个 HTML 和这个 iPython notebook 一起做为你的作业提交。  